# Memory

In [9]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history.add_user_message("hi!")
history.add_ai_message("whats up?")

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi! My name is Hieu")
memory.chat_memory.add_ai_message("whats up? Mine is AI")
memory.load_memory_variables({})

{'history': 'Human: hi! My name is Hieu\nAI: whats up? Mine is AI'}

In [10]:
import sys
sys.path.append('../src')
from setup import get_openai_model, enable_tracing
from langchain.chains import ConversationChain

from langchain.memory import ConversationBufferMemory

MODEL = get_openai_model()
enable_tracing('mem_chat')
conversation = ConversationChain(
    llm=MODEL,
    verbose=True,
    memory=memory
)

OPENAI ready
Enable tracing at mem_chat


In [11]:
q = "Do you remember my name. If so, write a short text about it"
ans = conversation.predict(input=q)
ans



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi! My name is Hieu
AI: whats up? Mine is AI
Human: Do you remember my name. If so, write a short text about it
AI:

> Finished chain.


" Yes, I remember your name is Hieu. It's a pleasure to meet you, Hieu. I'm looking forward to getting to know you better."

# Hand-Test

In [12]:
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("I just watched 2 videos yesterday")
memory.chat_memory.add_ai_message("Wow! What are they about?")
memory.chat_memory.add_user_message("The first one is about a cat doing a jump trick, but in the end he fell down. What's more supprising is that he still doing good after the fall.")
memory.chat_memory.add_user_message("The second one is about a bady lion hunted his first prey. It was a small rabbit, but the chase was intense.")

chat_bot = ConversationChain(llm=MODEL, verbose=True, memory=memory)

In [13]:
q = "Can you share your view on 2 videos I told"
ans = chat_bot.predict(input=q)
ans



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: I just watched 2 videos yesterday
AI: Wow! What are they about?
Human: The first one is about a cat doing a jump trick, but in the end he fell down. What's more supprising is that he still doing good after the fall.
Human: The second one is about a bady lion hunted his first prey. It was a small rabbit, but the chase was intense.
Human: Can you share your view on 2 videos I told
AI:

> Finished chain.


" That sounds like an interesting pair of videos! It's amazing that the cat was able to recover from the fall and still do the trick. It's also impressive that the baby lion was able to successfully hunt its first prey. It's a reminder of the power of nature and the strength of animals."

# Agent-Chat

In [14]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    )
]

In [16]:
prefix = """Have a conversation with a human, answering the following questions as best you can.
You have access to the following tools:"""
suffix = """Begin!"
{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)
memory = ConversationBufferMemory(memory_key="chat_history")
memory.chat_memory.add_user_message("I just watched 2 videos yesterday")
memory.chat_memory.add_ai_message("Wow! What are they about?")
memory.chat_memory.add_user_message("The first one is about a cat doing a jump trick, but in the end he fell down. What's more supprising is that he still doing good after the fall.")
memory.chat_memory.add_user_message("The second one is about a bady lion hunted his first prey. It was a small rabbit, but the chase was intense.")

In [19]:
llm_chain = LLMChain(llm=MODEL, prompt=prompt)

agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)

agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

In [22]:
q = "Can you remember what I have told you"
ans = agent_chain.run(input=q)
ans



> Entering new  chain...
Thought: I should try to recall the information
Action: Search
Action Input: Information about the first animal in the video
Observation: Do you ever wonder what the first animal on earth looked like? What did they do? How did they live? Join Stefan Chin for a fun new episode of SciShow that ta... Go to https://curiositystream.thld.co/historyoftheearth_1122 and use code HISTORYOFTHEEARTH to save 25% off today, that's only $14.99 a year. Thanks to Curio... Evolution The Closest Living Relative of the First Animal Has Finally Been Found A debate has been settled over the earliest animal ancestor—a free-swimming creature with a well-developed... BERKELEY, Calif. — Scientists believe they've identified the first animal to appear on Earth: a peculiar jellyfish-like creature. Researchers say all species evolved from a single common ancestor that lived as far back as 700 million years ago. These include creatures as diverse as sponges, insects, sharks, and humans. T

'The first animal in the video was a comb jellyfish.'

# Attempt 1: Load transcript from database to string and put it into memory 

In [1]:
videos = []
links = ['python_transcript.txt','redis_transcript.txt']
for link in links:
    f = open(f"data/{link}", "r")
    videos.append(f.read().replace('\n',''))
videos[0]

"python a high-level interpretedprogramming language famous for itszen-like code it's arguably the mostpopular language in the world becauseit's easy to learn yet practical forserious projects in fact you're watchingthis youtube video in a python webapplication right now it was created byguido van rossum and released in 1991who named it after monty python's flyingcircus which is why you'll sometimesfind spam and eggs instead of foo andbar in code samples it's commonly usedto build server-side applications likeweb apps with the django framework andis the language of choice for big dataanalysis and machine learning manystudents choose python to start learningto code because of its emphasis onreadability as outlined by the zen ofpython beautiful is better than uglywhile explicit is better than implicitpython is very simple but avoids thetemptation to sprinkle in magic thatcauses ambiguity its code is oftenorganized into notebooks whereindividual cells can be executed thendocumented in the

In [2]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("I just watched 2 videos yesterday")
memory.chat_memory.add_ai_message("Wow! What are they about?")
[memory.chat_memory.add_user_message(v) for v in videos]
chat_bot = ConversationChain(llm=MODEL, verbose=True, memory=memory)

NameError: name 'MODEL' is not defined

In [34]:
memory.chat_memory.messages[2]

HumanMessage(content="python a high-level interpretedprogramming language famous for itszen-like code it's arguably the mostpopular language in the world becauseit's easy to learn yet practical forserious projects in fact you're watchingthis youtube video in a python webapplication right now it was created byguido van rossum and released in 1991who named it after monty python's flyingcircus which is why you'll sometimesfind spam and eggs instead of foo andbar in code samples it's commonly usedto build server-side applications likeweb apps with the django framework andis the language of choice for big dataanalysis and machine learning manystudents choose python to start learningto code because of its emphasis onreadability as outlined by the zen ofpython beautiful is better than uglywhile explicit is better than implicitpython is very simple but avoids thetemptation to sprinkle in magic thatcauses ambiguity its code is oftenorganized into notebooks whereindividual cells can be executed 

In [35]:
q = "What was the first video about?"
ans = chat_bot.predict(input=q)
ans



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: I just watched 2 videos yesterday
AI: Wow! What are they about?
Human: python a high-level interpretedprogramming language famous for itszen-like code it's arguably the mostpopular language in the world becauseit's easy to learn yet practical forserious projects in fact you're watchingthis youtube video in a python webapplication right now it was created byguido van rossum and released in 1991who named it after monty python's flyingcircus which is why you'll sometimesfind spam and eggs instead of foo andbar in code samples it's commonly usedto build server-side applications likeweb apps with the django framework andis the language of choice for big dataanalysis and machine learn

' The first video was about the Python programming language. It discussed its history, features, and syntax. It also highlighted its popularity and ease of use, as well as its use in server-side applications, big data analysis, and machine learning.'

In [36]:
q = "Do you think that two technologies in those videos will be forever popular?"
ans = chat_bot.predict(input=q)
ans



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: I just watched 2 videos yesterday
AI: Wow! What are they about?
Human: python a high-level interpretedprogramming language famous for itszen-like code it's arguably the mostpopular language in the world becauseit's easy to learn yet practical forserious projects in fact you're watchingthis youtube video in a python webapplication right now it was created byguido van rossum and released in 1991who named it after monty python's flyingcircus which is why you'll sometimesfind spam and eggs instead of foo andbar in code samples it's commonly usedto build server-side applications likeweb apps with the django framework andis the language of choice for big dataanalysis and machine learn

" I don't know. It's hard to predict the future popularity of any technology. However, Python and Redis have both been around for a while and have been widely adopted, so it's likely that they will remain popular for some time."

## Past day conversation

In [38]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

videos = []
links = ['python_transcript.txt','redis_transcript.txt']
for link in links:
    f = open(f"data/{link}", "r")
    videos.append(f.read().replace('\n',''))

memory = ConversationBufferMemory()
for day, video in enumerate(videos,1):
    print(day)
    memory.chat_memory.add_user_message(f"The video that I watched on day {day}")
    memory.chat_memory.add_user_message(f"The video is about: {video}")

chat_bot = ConversationChain(llm=MODEL, verbose=True, memory=memory)

1
2


In [39]:
q = "What did I watch on day 1 and can you tell what is my job based on that"
ans = chat_bot.predict(input=q)
ans



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: The video that I watched on day 1
Human: The video is about: python a high-level interpretedprogramming language famous for itszen-like code it's arguably the mostpopular language in the world becauseit's easy to learn yet practical forserious projects in fact you're watchingthis youtube video in a python webapplication right now it was created byguido van rossum and released in 1991who named it after monty python's flyingcircus which is why you'll sometimesfind spam and eggs instead of foo andbar in code samples it's commonly usedto build server-side applications likeweb apps with the django framework andis the language of choice for big dataanalysis and machine learning manyst

' On day 1, you watched a video about Python, a high-level interpreted programming language. Based on the video, it appears that you are interested in programming and may have a job related to coding.'